<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#compare performance of different selection methods, crossovers, and mutations

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [1]:
import sys
sys.path.append('..')

In [2]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations
from typing import Callable
from library.algorithms.genetic_algorithms.algorithm import get_best_ind
from library.algorithms.genetic_algorithms.mutation import inversion_mutation

In [3]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('data/seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [4]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('data/seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]], shape=(64, 64))

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [5]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                #while any(idx in table for table in tables):
                    #idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self, verbose=False):
        total_happiness = 0
        i = 1
        for table in self.repr:
            # Sum relationship scores for all unique guest pairs at the table
            table_happiness = sum(self.relationship_matrix[a, b] for a, b in combinations(table, 2))
            total_happiness += table_happiness
            if verbose:
                print(f"Table {i} Happiness = {table_happiness}")
                i += 1
        return total_happiness

In [6]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness(verbose=True))

Random solution: [[6, 7, 62, 21, 25, 45, 50, 4], [9, 34, 22, 63, 2, 43, 20, 54], [10, 18, 29, 46, 44, 31, 38, 15], [55, 19, 51, 33, 17, 40, 61, 59], [53, 57, 28, 35, 48, 36, 41, 0], [42, 60, 32, 27, 24, 37, 16, 13], [49, 23, 3, 56, 8, 39, 52, 11], [5, 30, 12, 58, 1, 14, 26, 47]]
Table 1 Happiness = 2000.0
Table 2 Happiness = 300.0
Table 3 Happiness = 400.0
Table 4 Happiness = 300.0
Table 5 Happiness = 300.0
Table 6 Happiness = 900.0
Table 7 Happiness = 300.0
Table 8 Happiness = 2400.0
Total Fitness: 6900.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### 3.1. Fitness Proportionate Selection

In a typical **fitness proportionate selection** (also known as **roulette wheel selection**), individuals are selected based on their fitness values, with higher fitness values having a greater chance of being chosen. In our optimization problem, however, the fitness values can range from **negative** (indicating conflicts) to **positive** (indicating good relationships between guests).

To adapt the selection process to handle **negative fitness values**, we introduce the following modifications:

1. **Handling Negative Fitness**:
   - Negative fitness values (e.g., -1000) represent undesirable relationships between guests and are valid in our case.
   - However, to ensure that these negative values don't completely block a solution from being selected, we **normalize** the fitness values by adding an **offset** to make all fitness values non-negative.
   - The offset is the **absolute value of the minimum fitness** if it's negative, ensuring that all fitness values become positive or zero without altering the relative fitness ranking of solutions.

2. **Fitness Normalization**:
   - For each individual, we add the **offset** (if necessary) to their fitness value to ensure it's non-negative.
   - This ensures that individuals with negative fitness still have a chance to be selected but are less likely to be chosen compared to those with higher (positive) fitness.

3. **Roulette Wheel Selection**:
   - After normalizing the fitness values, the selection process proceeds as usual, with higher fitness solutions having a higher probability of being selected.
   - By adding the offset, we make sure that **negative fitness values** don't result in zero or near-zero chances of selection, allowing them to contribute to the evolutionary process.

In [7]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    if maximization:
        fitness_values = []
        min_fitness = min(ind.fitness() for ind in population)  # Find the minimum fitness value
        
        # Add a constant offset to ensure no negative fitness values
        offset = abs(min_fitness) if min_fitness < 0 else 0
        
        # Add the offset to each fitness value
        for ind in population:
            fitness_value = ind.fitness() + offset
            fitness_values.append(fitness_value)
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    
    # For each individual, check if random number is inside the individual's "box"
    box_boundary = 0
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

In [8]:
population = [WSSolution() for _ in range(10)]

fitness_proportionate_selection_result = fitness_proportionate_selection(population, maximization=True)
print('Fitness Proportionate selection result:', fitness_proportionate_selection_result)

Fitness Proportionate selection result: [[47, 10, 25, 15, 14, 33, 9, 56], [39, 40, 13, 21, 26, 29, 42, 12], [35, 18, 20, 55, 49, 62, 28, 11], [34, 45, 61, 23, 38, 52, 3, 41], [24, 48, 31, 19, 36, 37, 57, 51], [27, 8, 7, 30, 2, 32, 4, 50], [0, 60, 53, 44, 58, 22, 59, 6], [43, 63, 16, 46, 1, 54, 5, 17]]


#### 3.2. Ranking Selection

With Ranking Selection, instead of selecting based on raw fitness values, individuals are ranked by their fitness, and higher-ranked individuals (better seating arrangements) have a higher chance of being selected. This method ensures that negative fitness values aren´t automatically excluded, as selection is based on relative rank rather than absolute fitness. It maintains diversity by giving lower-ranked individuals a chance to contribute, preventing premature convergence.

In [27]:
def ranking_selection(population, maximization=False):
    n_parents = 1  # Always select one parent per call in the GA

    # Sort population based on fitness
    sorted_pop = sorted(
        population,
        key=lambda sol: sol.fitness(),
        reverse=maximization  # reverse=True if maximizing
    )

    # Assign linear ranks
    n = len(sorted_pop)
    ranks = np.arange(1, n + 1)
    total_rank = np.sum(ranks)
    probabilities = ranks / total_rank

    # If maximizing, best is first → reverse probabilities so high rank = high prob
    if maximization:
        probabilities = probabilities[::-1]

    selected = np.random.choice(sorted_pop, size=n_parents, p=probabilities, replace=True)
    return selected[0]  # Return single selected parent


In [28]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")


Fitness of individuals in population:
Individual 0: Fitness = 10500.00
Individual 1: Fitness = 18600.00
Individual 2: Fitness = 9100.00
Individual 3: Fitness = 17200.00
Individual 4: Fitness = 13000.00
Individual 5: Fitness = 13100.00
Individual 6: Fitness = 13500.00
Individual 7: Fitness = 19700.00
Individual 8: Fitness = 10700.00
Individual 9: Fitness = 16900.00

Ranking selection result:
Selected 1: [[38, 34, 18, 13, 19, 59, 50, 56], [39, 52, 45, 5, 21, 0, 43, 17], [14, 32, 60, 62, 3, 26, 25, 47], [57, 54, 46, 35, 55, 2, 49, 37], [23, 31, 29, 27, 24, 9, 40, 48], [30, 28, 20, 6, 61, 41, 58, 4], [11, 22, 16, 10, 44, 51, 33, 63], [8, 7, 12, 42, 1, 53, 36, 15]], Fitness = 18600.00
Selected 2: [[38, 34, 18, 13, 19, 59, 50, 56], [39, 52, 45, 5, 21, 0, 43, 17], [14, 32, 60, 62, 3, 26, 25, 47], [57, 54, 46, 35, 55, 2, 49, 37], [23, 31, 29, 27, 24, 9, 40, 48], [30, 28, 20, 6, 61, 41, 58, 4], [11, 22, 16, 10, 44, 51, 33, 63], [8, 7, 12, 42, 1, 53, 36, 15]], Fitness = 18600.00


In this case, we can see that the same individual (2nd best in rank) was selected twice. This is okay and makes that it coud happen especially with the high rank and small population.

In [30]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")

Fitness of individuals in population:
Individual 0: Fitness = 11600.00
Individual 1: Fitness = 14000.00
Individual 2: Fitness = 10500.00
Individual 3: Fitness = 23800.00
Individual 4: Fitness = 12300.00
Individual 5: Fitness = 12900.00
Individual 6: Fitness = 6400.00
Individual 7: Fitness = 11300.00
Individual 8: Fitness = 15400.00
Individual 9: Fitness = 14600.00

Ranking selection result:
Selected 1: [[22, 23, 36, 37, 2, 56, 49, 26], [17, 47, 41, 14, 40, 6, 57, 21], [1, 55, 7, 32, 9, 5, 38, 11], [52, 63, 0, 50, 16, 51, 59, 39], [3, 46, 34, 10, 48, 4, 60, 42], [54, 62, 31, 44, 53, 28, 27, 25], [15, 19, 45, 58, 35, 18, 13, 33], [30, 43, 24, 8, 12, 61, 29, 20]], Fitness = 14000.00
Selected 2: [[53, 14, 52, 50, 1, 28, 51, 11], [36, 9, 45, 56, 0, 57, 2, 30], [42, 25, 39, 33, 58, 8, 60, 10], [47, 7, 48, 35, 3, 6, 29, 32], [15, 38, 16, 40, 24, 23, 63, 12], [17, 34, 49, 62, 44, 13, 21, 55], [31, 54, 20, 27, 41, 19, 5, 59], [46, 43, 61, 26, 22, 37, 4, 18]], Fitness = 23800.00


#### 3.3. Tournament Selection

In [11]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

In [12]:
population = [WSSolution() for _ in range(10)]

tournament_selection_result = tournament_selection(population)
print('Tournament selection result:', tournament_selection_result)

Tournament selection result: [[31, 57, 14, 17, 55, 22, 4, 23], [46, 29, 20, 60, 11, 44, 37, 1], [48, 27, 12, 26, 21, 49, 34, 58], [2, 0, 15, 32, 19, 9, 47, 54], [16, 35, 3, 56, 50, 52, 51, 42], [62, 6, 40, 63, 33, 36, 7, 53], [39, 13, 24, 41, 18, 43, 59, 25], [8, 38, 30, 10, 61, 5, 45, 28]]


---

### <span style="color:#96d9f0;">4. Crossover Operators</span>

#### 4.1. Custom Crossover Operator (`table_level_crossover`)

In this section, we implement a custom crossover operator tailored to the constraints of our optimization problem. Unlike standard one-point crossover, which is suitable for flat representations, our solution requires a structured approach:

- Each solution is a nested list representing 8 tables with 8 unique guests each.
- Every guest (from 0 to 63) must appear exactly once across all tables.
- A direct crossover risks creating invalid offspring (with duplicated or missing guests).

To address this, our operator performs table-level crossover:

1. A one-point crossover is applied between the two parent solutions at the table level. This can result in invalid intermediate offspring that contain duplicate guests and omit others, violating the uniqueness constraint.

2. The resulting child is then repaired by:

    - Detecting duplicate and missing guests
    - Replacing duplicates with missing ones to restore a valid configuration
    - This ensures all offspring are valid solutions and suitable for use in the genetic algorithm.

In [13]:
def table_level_crossover(parent1, parent2):
    # Deep copies of the parent representations
    p1_repr = deepcopy(parent1.repr)
    p2_repr = deepcopy(parent2.repr)

    # Flatten guest lists
    total_guests = len(parent1.relationship_matrix)

    # One-point crossover at table level
    xo_point = random.randint(1, len(p1_repr) - 1)
    child1_repr = p1_repr[:xo_point] + p2_repr[xo_point:]
    child2_repr = p2_repr[:xo_point] + p1_repr[xo_point:]

    def repair(child_repr):
        # Count how many times each guest appears
        guest_counts = {}
        for table in child_repr:
            for guest in table:
                guest_counts[guest] = guest_counts.get(guest, 0) + 1

        # Identify missing and extra guests
        missing = [g for g in range(total_guests) if guest_counts.get(g, 0) == 0]
        extras = [g for g, count in guest_counts.items() if count > 1 for _ in range(count - 1)]

        # Replace extra guests with missing ones
        for table in child_repr:
            for i in range(len(table)):
                guest = table[i]
                if guest_counts[guest] > 1:
                    new_guest = missing.pop()
                    guest_counts[guest] -= 1
                    table[i] = new_guest
                    guest_counts[new_guest] = 1  # added to the table

        return child_repr

    # Repair both children
    repaired1 = repair(child1_repr)
    repaired2 = repair(child2_repr)

    return WSSolution(repr=repaired1), WSSolution(repr=repaired2)

In [14]:
solution1 = WSSolution()
solution2 = WSSolution()

table_level_crossover_result = table_level_crossover(solution1, solution2)

print('Parent 1:', solution1)
print('Parent 2:', solution2)
print('Offspring 1:', table_level_crossover_result[0])
print('Offspring 2:', table_level_crossover_result[1])

Parent 1: [[13, 16, 61, 10, 41, 30, 1, 0], [32, 6, 17, 60, 14, 57, 20, 29], [4, 62, 40, 59, 38, 54, 52, 12], [50, 63, 27, 25, 31, 22, 35, 36], [47, 44, 28, 11, 48, 55, 46, 26], [9, 34, 2, 3, 5, 21, 24, 37], [18, 45, 7, 53, 15, 49, 19, 33], [56, 51, 43, 58, 39, 42, 8, 23]]
Parent 2: [[29, 45, 36, 27, 12, 49, 24, 32], [61, 59, 3, 46, 63, 26, 18, 42], [43, 57, 14, 34, 56, 5, 7, 19], [37, 53, 60, 47, 41, 8, 16, 30], [2, 48, 62, 52, 55, 11, 17, 31], [51, 50, 44, 13, 1, 58, 39, 40], [21, 22, 23, 33, 20, 9, 25, 38], [35, 6, 0, 15, 28, 10, 4, 54]]
Offspring 1: [[49, 45, 36, 32, 29, 27, 24, 12], [61, 59, 3, 46, 63, 26, 18, 42], [43, 57, 14, 34, 56, 5, 7, 19], [37, 53, 60, 47, 41, 8, 16, 30], [2, 48, 62, 52, 55, 11, 17, 31], [51, 50, 44, 13, 1, 58, 39, 40], [21, 22, 23, 33, 20, 9, 25, 38], [35, 6, 0, 15, 28, 10, 4, 54]]
Offspring 2: [[61, 41, 30, 16, 13, 10, 1, 0], [32, 6, 17, 60, 14, 57, 20, 29], [4, 62, 40, 59, 38, 54, 52, 12], [50, 63, 27, 25, 31, 22, 35, 36], [47, 44, 28, 11, 48, 55, 46, 26]

#### 4.2. Partially Matched Crossover

In [45]:
def partially_matched_crossover(parent1, parent2):
    # Flatten parent representations (PMC works on 1D permutations)
    def flatten(tables):
        return [guest for table in tables for guest in table]
    
    # Function to unflatten the list back to the original representation, after crossover
    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    table_size = parent1.table_capacity
    total_guests = len(parent1.relationship_matrix)

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(total_guests), 2))

    def pmc(parent1, parent2):
        child = [None] * total_guests
        # Copy crossover segment from parent2
        child[point1:point2] = parent2[point1:point2]
        
        # Set to track already placed guests in child
        placed_guests = set(child[point1:point2])

        # Mapping for conflict resolution
        for i in range(point1, point2):
            if parent1[i] not in placed_guests:
                val = parent1[i]
                pos = i
                # Resolve conflict by finding non-conflicting guests
                while parent2[pos] in placed_guests:
                    pos = parent1.index(parent2[pos])
                child[pos] = val
                placed_guests.add(val)

        # Fill remaining positions with non-conflicting genes
        for i in range(total_guests):
            if child[i] is None:
                # If the position is still None, place the corresponding guest from parent1
                if parent1[i] not in placed_guests:
                    child[i] = parent1[i]
                    placed_guests.add(parent1[i])

        return child

The Partially Matched Crossover (pmc) is a genetic algorithm operator designed for permutations, like our guest seating arrangement. It swaps a segment between two parent solutions and builds offspring by preserving the order and position of elements while avoiding duplicates. In our implementation, we adapted PMX to work with the wedding seating format by flattening table arrangements into 1D lists, applying PMX, and then converting them back. We carefully resolved conflicts during crossover by mapping duplicate entries to valid unused guests, ensuring that each guest appears exactly once in the final seating—maintaining valid, non-repetitive arrangements.

In [52]:
def partially_matched_crossover(parent1, parent2):
    """
    PMX crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    import random

    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def pmc(p1, p2):
        child = [None] * size
        # Copy the crossover segment from p1
        child[point1:point2] = p1[point1:point2]

        for i in range(point1, point2):
            if p2[i] not in child:
                val = p2[i]
                idx = i
                while True:
                    mapped = p1[idx]
                    idx = p2.index(mapped)
                    if child[idx] is None:
                        child[idx] = val
                        break

        # Fill remaining positions from p2
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]

        return child

    child1_flat = pmc(parent1_flat, parent2_flat)
    child2_flat = pmc(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2


In [53]:
# Create 2 solutions
parent1 = WSSolution(nr_of_tables=4)
parent2 = WSSolution(nr_of_tables=4)

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = partially_matched_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[26, 55, 51, 14, 48, 0, 63, 29, 12, 18, 44, 23, 36, 24, 16, 8], [20, 34, 53, 19, 61, 6, 1, 58, 9, 39, 30, 52, 45, 33, 40, 50], [5, 3, 7, 28, 11, 49, 10, 62, 4, 22, 60, 38, 43, 46, 37, 35], [25, 13, 21, 17, 56, 42, 47, 2, 31, 59, 41, 27, 15, 32, 54, 57]]
Parent 2 Seating Arrangement:
[[35, 41, 10, 11, 43, 63, 29, 31, 46, 16, 54, 62, 22, 8, 18, 1], [30, 45, 38, 9, 53, 50, 56, 49, 12, 26, 15, 34, 51, 3, 47, 24], [57, 59, 19, 48, 40, 39, 44, 17, 7, 25, 4, 23, 33, 21, 55, 36], [32, 37, 6, 27, 13, 28, 60, 2, 0, 58, 14, 5, 52, 20, 42, 61]]

Offspring 1 Seating Arrangement:
[[35, 41, 10, 11, 43, 31, 63, 29, 12, 18, 44, 23, 36, 24, 16, 8], [20, 34, 53, 19, 61, 6, 56, 49, 46, 26, 15, 45, 51, 3, 47, 1], [57, 59, 9, 48, 40, 39, 54, 17, 7, 25, 4, 62, 33, 21, 55, 22], [32, 37, 50, 27, 13, 28, 60, 2, 0, 58, 14, 5, 52, 30, 42, 38]]
Offspring 2 Seating Arrangement:
[[26, 55, 51, 14, 48, 0, 29, 31, 46, 16, 54, 62, 22, 8, 18, 1], [30, 45, 38, 9, 53, 50, 24, 58, 19, 39, 20, 

In [54]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.3. Group Preserving Crossover

In [19]:
# Code (Pedro)

---

### <span style="color:#96d9f0;">5. Mutation Operators</span>

#### 5.1. Swap Mutation

The swap mutation operator introduces random changes to the seating arrangement by swapping two guests between different tables.

It's important to note that swapping two guests within the same table would not create any new configuration - it would simply result in the same solution as before. Since we want to introduce meaningful changes, the swap mutation ensures that the two guests are always swapped between different tables. This prevents the mutation from generating an invalid or identical solution.

This mutation helps the genetic algorithm explore new seating arrangements by shuffling guests across tables while respecting the overall constraints of the problem.

In [17]:
def swap_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    if len(tables) < 2:
        return new_solution

    # Pick two random tables
    t1, t2 = random.sample(range(len(tables)), 2)
    if not tables[t1] or not tables[t2]:
        return new_solution

    # Swap one person from each
    i1 = random.randint(0, len(tables[t1]) - 1)
    i2 = random.randint(0, len(tables[t2]) - 1)
    tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

    return new_solution

In [18]:
solution = WSSolution()

mutated_solution = swap_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[21, 23, 62, 49, 41, 30, 50, 48], [28, 4, 0, 42, 31, 11, 39, 32], [29, 45, 51, 8, 35, 34, 20, 9], [37, 10, 5, 7, 19, 43, 61, 58], [44, 56, 22, 2, 18, 59, 15, 38], [12, 52, 24, 17, 36, 60, 3, 16], [33, 26, 63, 57, 1, 25, 14, 40], [54, 6, 13, 47, 55, 27, 46, 53]]
Mutated solution: [[21, 23, 62, 49, 41, 10, 50, 48], [28, 4, 0, 42, 31, 11, 39, 32], [29, 45, 51, 8, 35, 34, 20, 9], [37, 30, 5, 7, 19, 43, 61, 58], [44, 56, 22, 2, 18, 59, 15, 38], [12, 52, 24, 17, 36, 60, 3, 16], [33, 26, 63, 57, 1, 25, 14, 40], [54, 6, 13, 47, 55, 27, 46, 53]]


#### 5.2. Inversion Mutation

The inversion mutation operator introduces variation by selecting a random segment of the seating plan and reversing the order of guests within that segment.

Since the seating arrangement is represented as a list of tables, the entire structure is first flattened into a single list containing all guests. The inversion is then performed on a random sublist, effectively inverting a continuous sequence of guests. After the inversion, the list is reshaped back into the original table format.

This mutation allows the algorithm to explore new configurations by reordering guests while preserving the structure of the solution. To ensure validity, a repair function is applied after mutation to guarantee that all guests remain uniquely assigned with no duplicates or omissions.

In [19]:
def inversion_mutation_seating_Afonso(individual: WSSolution, mut_prob):
    tables = individual.repr  # Access the 8x8 seating arrangement

    # Flatten the 8x8 seating into a list of 64 guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation
    mutated_flat = inversion_mutation(flat, mut_prob)

    # Reshape back to 8 tables of 8
    new_tables = [mutated_flat[i:i+8] for i in range(0, 64, 8)]

    # Return a new WSSolution with the mutated tables
    return WSSolution(repr=new_tables)

In [20]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating_Afonso(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[44, 15, 4, 19, 24, 55, 8, 50], [25, 35, 46, 18, 5, 34, 7, 2], [41, 12, 16, 60, 32, 54, 58, 3], [26, 62, 47, 30, 28, 11, 14, 0], [9, 27, 21, 1, 40, 31, 43, 61], [51, 10, 48, 17, 38, 23, 6, 22], [20, 33, 29, 13, 56, 39, 59, 36], [53, 45, 49, 63, 37, 42, 52, 57]]
Mutated solution: [[44, 15, 4, 19, 24, 55, 8, 50], [25, 34, 5, 18, 46, 35, 7, 2], [41, 12, 16, 60, 32, 54, 58, 3], [26, 62, 47, 30, 28, 11, 14, 0], [9, 27, 21, 1, 40, 31, 43, 61], [51, 10, 48, 17, 38, 23, 6, 22], [20, 33, 29, 13, 56, 39, 59, 36], [53, 45, 49, 63, 37, 42, 52, 57]]


In [21]:
def inversion_mutation_seating_Rita(individual: WSSolution, mut_prob):
    # Access the seating arrangement (8 tables with guests)
    tables = individual.repr
    # Flatten the seating arrangement into a single list of guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation with the given probability
    if random.random() < mut_prob:
        # Select two random indices for the inversion
        start, end = sorted(random.sample(range(len(flat)), 2))
        # Reverse the segment between the two indices
        flat[start:end + 1] = reversed(flat[start:end + 1])

    # Reshape the flat list back into 8 tables of 8 guests each
    new_tables = [flat[i:i + individual.table_capacity] for i in range(0, len(flat), individual.table_capacity)]

    # Return a new WSSolution with the mutated seating arrangement
    return WSSolution(repr=new_tables, relationship_matrix=individual.relationship_matrix, nr_of_tables=individual.nr_of_tables)

In [25]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating_Rita(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[1, 20, 11, 45, 16, 56, 36, 41], [19, 28, 2, 29, 9, 51, 43, 12], [0, 25, 22, 53, 49, 30, 8, 60], [59, 37, 48, 61, 38, 47, 58, 31], [50, 35, 55, 15, 39, 23, 10, 24], [44, 52, 7, 6, 57, 42, 27, 18], [46, 63, 21, 33, 4, 17, 40, 14], [26, 54, 3, 32, 13, 34, 5, 62]]
Mutated solution: [[1, 20, 11, 45, 16, 56, 36, 41], [19, 28, 2, 29, 9, 51, 43, 12], [0, 25, 22, 53, 49, 30, 8, 32], [3, 54, 26, 14, 40, 17, 4, 33], [21, 63, 46, 18, 27, 42, 57, 6], [7, 52, 44, 24, 10, 23, 39, 15], [55, 35, 50, 31, 58, 47, 38, 61], [48, 37, 59, 60, 13, 34, 5, 62]]


#### 5.3. New Creative Mutation

In [26]:
# Code (someone)

---

### <span style="color:#96d9f0;">6. Genetic Algorithm</span>

In [22]:
def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    crossover_operator: Callable,
    mutation_operator: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        crossover_operator (Callable): Function to perform crossover between two individuals.
        mutation_operator (Callable): Function to mutate an individual.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found after evolving for max_gen generations.
    """
    population = initial_population

    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        new_population = []

        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))

        while len(new_population) < len(population):
            parent1 = selection_algorithm(population, maximization)
            parent2 = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected parents:\n{parent1}\n{parent2}')

            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                if verbose:
                    print('Crossover applied.')
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)
                if verbose:
                    print('Replication applied.')

            mutated1 = mutation_operator(offspring1, mut_prob)
            new_population.append(mutated1)
            if verbose:
                print(f'Mutated offspring 1: {mutated1}')

            if len(new_population) < len(population):
                mutated2 = mutation_operator(offspring2, mut_prob)
                new_population.append(mutated2)
                if verbose:
                    print(f'Mutated offspring 2: {mutated2}')

        population = new_population

        if verbose:
            best = get_best_ind(population, maximization)
            print(f'Best individual fitness this generation: {best.fitness()}')

    return get_best_ind(population, maximization)

In [63]:
# Number of individuals in the initial population
population_size = 50

# Generate initial population of random WSSolutions
population = [WSSolution() for _ in range(population_size)]

In [29]:
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best)
print("Fitness:", best.fitness())

-------------- Generation: 1 --------------
Selected parents:
[[60, 41, 34, 46, 33, 6, 28, 15], [61, 45, 1, 39, 36, 47, 57, 4], [52, 23, 26, 10, 29, 19, 13, 54], [56, 5, 27, 55, 22, 58, 17, 35], [49, 38, 7, 21, 44, 51, 0, 9], [31, 48, 59, 20, 25, 63, 42, 18], [2, 43, 16, 3, 53, 50, 62, 37], [14, 12, 32, 30, 11, 40, 24, 8]]
[[59, 44, 13, 33, 39, 60, 29, 26], [1, 3, 38, 63, 19, 2, 22, 17], [52, 20, 61, 41, 0, 46, 47, 11], [40, 14, 30, 34, 4, 6, 32, 21], [12, 50, 15, 54, 35, 18, 56, 57], [27, 36, 24, 5, 45, 51, 10, 53], [37, 7, 31, 16, 42, 43, 23, 25], [58, 62, 28, 9, 8, 55, 48, 49]]
Crossover applied.
Mutated offspring 1: [[60, 63, 59, 44, 33, 38, 29, 26], [22, 19, 1, 39, 17, 13, 3, 2], [52, 20, 61, 41, 0, 46, 47, 11], [40, 14, 30, 34, 4, 6, 32, 21], [12, 50, 15, 54, 35, 18, 56, 57], [27, 36, 24, 5, 45, 51, 10, 53], [37, 7, 31, 16, 42, 43, 23, 25], [58, 62, 28, 9, 8, 55, 48, 49]]
Mutated offspring 2: [[61, 57, 47, 33, 39, 60, 46, 45], [1, 41, 36, 34, 28, 15, 6, 4], [52, 23, 26, 10, 29, 1

KeyboardInterrupt: 

In [ ]:
isa_try = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(isa_try)
print("Fitness:", isa_try.fitness())

In [21]:
# Number of individuals in the initial population
population_size2 = 3

# Generate initial population of random WSSolutions
population2 = [WSSolution() for _ in range(population_size2)]

best2 = genetic_algorithm(
    initial_population=population2,
    max_gen=5,
    selection_algorithm=tournament_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=inversion_mutation_seating,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best2)
print("Fitness:", best2.fitness())

-------------- Generation: 1 --------------
Selected parents:
[[46, 22, 14, 45, 2, 55, 61, 7], [17, 16, 10, 6, 37, 11, 0, 4], [36, 41, 30, 12, 31, 29, 21, 47], [1, 39, 40, 5, 38, 3, 49, 62], [48, 58, 35, 27, 32, 33, 63, 18], [23, 59, 52, 34, 9, 43, 25, 54], [53, 57, 28, 13, 42, 8, 56, 50], [19, 24, 60, 15, 44, 51, 26, 20]]
[[54, 13, 46, 35, 51, 30, 1, 62], [26, 9, 14, 19, 8, 27, 5, 20], [45, 2, 56, 18, 50, 17, 58, 28], [24, 3, 59, 6, 16, 40, 34, 57], [7, 4, 53, 49, 33, 44, 12, 32], [29, 10, 36, 48, 31, 61, 39, 47], [22, 11, 0, 60, 63, 42, 55, 15], [25, 21, 37, 41, 52, 38, 43, 23]]
Crossover applied.
Mutated offspring 1: [[46, 62, 54, 51, 35, 30, 13, 1], [26, 9, 14, 19, 8, 27, 5, 20], [45, 2, 56, 18, 50, 17, 58, 28], [24, 3, 59, 6, 16, 40, 34, 57], [7, 4, 53, 49, 33, 44, 12, 32], [29, 10, 36, 48, 31, 61, 39, 47], [22, 11, 0, 60, 63, 42, 55, 15], [25, 21, 37, 41, 52, 38, 43, 23]]
Mutated offspring 2: [[61, 55, 46, 45, 22, 14, 7, 2], [17, 16, 10, 6, 37, 11, 0, 4], [36, 41, 30, 12, 31, 29,

In [58]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
pmc_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(pmc_test.repr)
print("Fitness:", pmc_test.fitness())

-------------- Generation: 1 --------------
Selected parents:
[[6, 53, 49, 8, 42, 2, 34, 25], [9, 28, 15, 62, 61, 11, 10, 30], [39, 32, 40, 27, 0, 55, 18, 22], [43, 24, 35, 29, 1, 58, 56, 14], [4, 36, 37, 12, 19, 41, 54, 5], [16, 23, 3, 20, 51, 13, 57, 7], [60, 63, 59, 38, 17, 48, 45, 46], [44, 26, 52, 21, 33, 47, 31, 50]]
[[56, 42, 47, 19, 22, 6, 17, 55], [10, 41, 57, 52, 31, 33, 53, 5], [16, 8, 40, 37, 51, 23, 34, 4], [38, 21, 35, 25, 24, 0, 43, 18], [20, 30, 61, 48, 28, 36, 11, 26], [3, 63, 7, 50, 60, 32, 27, 12], [59, 1, 62, 29, 46, 15, 45, 13], [58, 39, 49, 2, 14, 54, 9, 44]]
Replication applied.
Mutated offspring 1: [[6, 53, 49, 8, 42, 2, 34, 25], [9, 28, 15, 62, 61, 11, 10, 30], [39, 32, 40, 27, 0, 55, 18, 22], [43, 24, 35, 29, 1, 58, 56, 14], [4, 36, 37, 12, 19, 41, 54, 5], [16, 23, 3, 20, 51, 13, 57, 7], [60, 63, 59, 38, 17, 48, 45, 46], [44, 26, 52, 21, 33, 47, 31, 50]]
Mutated offspring 2: [[56, 42, 47, 19, 22, 6, 17, 55], [10, 41, 57, 52, 31, 33, 53, 5], [16, 8, 40, 37, 51,

In [62]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(pmc_test.repr))

Offspring 2 has no repeated guests: True


In [60]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
ranking_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(ranking_test.repr)
print("Fitness:", ranking_test.fitness())

-------------- Generation: 1 --------------
Selected parents:
[[49, 8, 47, 52, 31, 0, 22, 23], [24, 32, 15, 59, 60, 19, 3, 26], [42, 5, 28, 25, 53, 16, 57, 33], [46, 9, 48, 58, 36, 17, 27, 7], [39, 20, 61, 2, 14, 51, 40, 63], [12, 37, 38, 10, 13, 44, 41, 54], [34, 6, 62, 21, 35, 11, 4, 45], [1, 55, 56, 30, 50, 43, 18, 29]]
[[50, 10, 11, 21, 17, 24, 47, 2], [60, 26, 53, 20, 19, 36, 27, 15], [30, 57, 38, 32, 49, 28, 59, 48], [40, 45, 12, 4, 0, 44, 34, 43], [41, 37, 6, 42, 55, 61, 29, 5], [39, 46, 13, 9, 31, 33, 14, 62], [58, 7, 35, 3, 22, 8, 54, 56], [63, 23, 51, 52, 18, 16, 1, 25]]
Crossover applied.
Mutated offspring 1: [[50, 10, 11, 21, 17, 0, 22, 23], [24, 32, 15, 59, 60, 19, 3, 26], [42, 5, 28, 25, 53, 16, 20, 48], [40, 45, 12, 4, 36, 44, 34, 43], [41, 37, 6, 30, 55, 61, 29, 57], [39, 46, 13, 9, 31, 33, 14, 62], [58, 7, 35, 27, 47, 8, 54, 56], [63, 2, 51, 52, 18, 38, 1, 49]]
Mutated offspring 2: [[25, 8, 22, 52, 31, 24, 47, 2], [60, 26, 53, 20, 19, 36, 27, 15], [30, 57, 38, 32, 49, 

In [61]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(ranking_test.repr))

Offspring 2 has no repeated guests: True
